# test of BM25s
BM25 Sparse
https://bm25s.github.io/

[ ] installed in venv bm25s

# Install with all extra dependencies
pip install bm25s[full]

# installed nltk for porter stemmer
pip install nltk


## documentation
https://github.com/xhluca/bm25s

In [1]:

import bm25s
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import os
import pickle




In [34]:
# Create your corpus here
## ********** change this to retrieve the data/documents.txt file for short term
## ********** long term: chanage this to query via SQL to DBeaver ofi database
input_dir = '/Users/poppyriddle/Documents/PhD/Research_proposal/Part_3/part_3_cohere/data'

#create list
corpus = []

url = []

#read each file in input_dir
for file_name in os.listdir(input_dir):
    if file_name.endswith('.txt'):
        file_path = os.path.join(input_dir,file_name)

        with open(file_path, 'r') as file:
            content = file.read()

            #extract title and abstract - I think we also need 'landing_page_url' for retrieval
            title = None
            abstract = None
            keyword = None
            doi = None

            for line in content.split('\n'):
                if line.startswith('DOI: '):
                    doi = line[len('DOI: '):].strip()
                if line.startswith('Title:'):
                    title = line[len('Title:'):].strip()
                elif line.startswith('Abstract:'):
                    abstract = line[len('Abstract:'):].strip()
                #elif line.startswith('keyword:'):
                #    keyword = line[len('keyword:'):].strip()
            
            if title and abstract:
                if keyword is not None:
                    #combine into single entry
                    document = f"{title} {abstract} {keyword}"
                    corpus.append(document)
                else:
                    document = f"{title} {abstract}"
                    corpus.append(document)

            #extract landing_page_url and combine with title
            #for line in content.split('\n'):
            #    if line.startswith('DOI'):
            #        doi = line[len('DOI'):].strip()

            if title and doi:
                resolver_doi = f"https://doi.org/{doi}"
                link = f"URL: {resolver_doi} for title: {title}"
                url.append(link)
                    


#export corpus and url lists for import later
with open('corpus.pkl', 'wb') as file:
    pickle.dump(corpus, file)

with open('url.pkl','wb') as file:
    pickle.dump(url, file)

print('complete')


complete


In [35]:
#optional stemmer
#stemmer = Stemmer.Stemmer("english")

#Tokenize the corpus and index it - removes stopwords 
#you can also add a stemmer here as an arg: stemmer=stemmer
corpus_tokens = bm25s.tokenize(corpus, stopwords="en", show_progress=True)

# Create the BM25 model and index the corpus
retriever = bm25s.BM25(corpus=corpus) # this arg returns docs, not ids. to have id's remove this arg
retriever.index(corpus_tokens)




In [36]:
# you can save the index

# You can save the corpus along with the model
retriever.save("data/bm25", corpus=corpus)

# ...and load them when you need them
retriever = bm25s.BM25.load("data/bm25", load_corpus=True)
# set load_corpus=False if you don't need the corpus

# reimport corpus and url lists
with open('corpus.pkl', 'rb') as file:
    corpus_list = pickle.load(file)
print(len(corpus_list))
with open('url.pkl', 'rb') as file:
    url_list = pickle.load(file)
print(len(url_list))


Finding newlines for mmindex: 100%|██████████| 59.0k/59.0k [00:00<00:00, 70.8MB/s]

43
43


In [37]:
#You can now search the corpus with a query
query = input("what is your query")
#you can also add a stemmer here as an arg: stemmer=stemmer
query_tokens = bm25s.tokenize(query,
                            stopwords=True,
                            lower=True)
results, scores = retriever.retrieve(query_tokens, k=3)

#note: if you pass a new corpus here, it must have the same length as your indexed corpus
#in this case, I am passing the new list 'url_list' - it contains just the DOI and title
results = retriever.retrieve(query_tokens, corpus=url_list, k=3, return_as="documents")
#loop through results
if all(score ==0.00 for score in scores[0]):
    print("Nothing found, please try another query.")
else:
    for i in range(results.shape[1]):
        doc, score = results[0, i], scores[0, i]
        print(f"Rank {i+1} (score: {score:.2f}): {doc}")



Rank 1 (score: 0.62): URL: https://doi.org/10.1162/qss_a_00022 for title: Crossref: The sustainable source of community-owned scholarly metadata
Rank 2 (score: 0.60): URL: https://doi.org/10.31222/osf.io/smxe5 for title: Crossref as a source of open bibliographic metadata
Rank 3 (score: 0.60): URL: https://doi.org/10.5860/crl.86.1.101 for title: Identifying Metadata Quality Issues Across Cultures


This is working using a dedicated virtual env.
- [ ] get this working in the same environment as app.py for the LNS project, 3.12 Python
- [ ] set up requirements
- [ ] turn into function module to import into app.py

In [1]:

import bm25s
import os
import pickle

In [2]:
# ...and load them when you need them
retriever = bm25s.BM25.load("lns_bm25", load_corpus=True)
# set load_corpus=False if you don't need the corpus

#load the url_list
with open('url.pkl', 'rb') as file:
    url_list = pickle.load(file)
print(len(url_list))

99


In [3]:
#You can now search the corpus with a query
query = input("what is your query")
#you can also add a stemmer here as an arg: stemmer=stemmer
query_tokens = bm25s.tokenize(query)
results, scores = retriever.retrieve(query_tokens, k=3)

#note: if you pass a new corpus here, it must have the same length as your indexed corpus
#in this case, I am passing the new list 'url_list'
results = retriever.retrieve(query_tokens, corpus=url_list, k=3, return_as="documents")
#loop through results
if all(score ==0.00 for score in scores[0]):
    print("Nothing found, please try another query.")
else:
    for i in range(results.shape[1]):
        doc, score = results[0, i], scores[0, i]
        print(f"Rank {i+1} (score: {score:.2f}): {doc}")

Rank 1 (score: 1.25): URL: : https://doi.org/10.1080/17408989.2021.2014438 for title: PLitPE: an intervention for physical literacy enriched pedagogy in Canadian elementary school physical education classes
Rank 2 (score: 1.24): URL: : https://doi.org/10.1186/s12889-018-5897-4 for title: The relationship between physical literacy scores and adherence to Canadian physical activity and sedentary behaviour guidelines
Rank 3 (score: 1.23): URL: : https://doi.org/10.1007/s11125-020-09519-5 for title: Increasing physical literacy in youth: A two-week Sport for Development program for children aged 6-10
